## Test xarray

- [x] check package versions with conda
- [x] import library
- [x] load data
- [x] resample

In [1]:
!mamba list

List of packages in environment: "/srv/conda/envs/notebook"

  Name                                  Version              Build                    Channel    
───────────────────────────────────────────────────────────────────────────────────────────────────
  _libgcc_mutex                         0.1                  conda_forge              conda-forge
  _openmp_mutex                         4.5                  2_gnu                    conda-forge
  adlfs                                 2024.12.0            pyhd8ed1ab_0             conda-forge
  adwaita-icon-theme                    48.0                 unix_0                   conda-forge
  affine                                2.4.0                pyhd8ed1ab_1             conda-forge
  aiobotocore                           2.22.0               pyhd8ed1ab_0             conda-forge
  aiohappyeyeballs                      2.6.1                pyhd8ed1ab_0             conda-forge
  aiohttp                               3.12.4         

In [2]:
import xarray as xr
import xarray.tutorial

air = xr.tutorial.load_dataset("air_temperature")
print(air)

<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...


In [3]:
monthly = air.air.resample({"time": "ME"}).mean()
print(monthly.min())

<xarray.DataArray 'air' ()> Size: 8B
array(231.72116071)


## Test geopandas

- [x] import library
- [x] read data from remote
- [x] render using shapely
- [x] render using folium

In [4]:
import geopandas as gpd

In [5]:
URL = "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_FRA_0.json"
df = gpd.read_file(URL)

In [6]:
df.geometry[0].bounds

(-5.1438, 41.3338, 9.5604, 51.0894)

## Test fsspec

- [x] import library
- [x] `ls` public s3 bucket

In [7]:
import fsspec

In [8]:
fs = fsspec.filesystem("s3", anon=True)

fs.ls("s3://sentinel-inventory/sentinel-s2-l1c/")

['sentinel-inventory/sentinel-s2-l1c/inventory-orc',
 'sentinel-inventory/sentinel-s2-l1c/sentinel-s2-l1c-inventory']

## Test PySTAC-client

- [x] import library
- [x] open catalog
- [x] search catalog

In [9]:
from pystac_client import Client

In [10]:
STAC_API_URL = "https://staging.openveda.cloud/api/stac/"
catalog = Client.open(STAC_API_URL)
print(catalog)  # don't want the full repr here since it will change everytime anything is added or removed from STAC

<Client id=stac-fastapi>


In [11]:
items = list(catalog.search(collections=["social-vulnerability-index-overall-nopop"], datetime="2000").items())
items

[<Item id=svi_2000_tract_overall_wgs84_nopop_cog>]

## Test hvplot

- [x] import library
- [x] geo plot
- [x] non-geo plot

In [12]:
import hvplot.xarray

In [13]:
p = monthly.hvplot(geo=True, x="lon", y="lat", dynamic=False)
print(p)

:HoloMap   [time]
   :Image   [lon,lat]   (4xDaily Air temperature at sigma level 995)


In [14]:
# Data that goes into plot for May
print(p.values()[4].data)

<xarray.Dataset> Size: 11kB
Dimensions:  (lat: 25, lon: 53, time: 1)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 8B 2013-05-31
Data variables:
    air      (time, lat, lon) float64 11kB 266.6 266.8 266.8 ... 297.1 297.1


In [15]:
p = monthly.sel(lat=40, lon=285).hvplot()
print(p)

:Curve   [time]   (4xDaily Air temperature at sigma level 995)


In [16]:
print(p.data)

             lat    lon         air
time                               
2013-01-31  40.0  285.0  274.552661
2013-02-28  40.0  285.0  273.603929
2013-03-31  40.0  285.0  276.138871
2013-04-30  40.0  285.0  282.911333
2013-05-31  40.0  285.0  288.135484
2013-06-30  40.0  285.0  293.957167
2013-07-31  40.0  285.0  297.506210
2013-08-31  40.0  285.0  295.102258
2013-09-30  40.0  285.0  291.740083
2013-10-31  40.0  285.0  288.149839
2013-11-30  40.0  285.0  279.911917
2013-12-31  40.0  285.0  276.186048
2014-01-31  40.0  285.0  270.596210
2014-02-28  40.0  285.0  272.124018
2014-03-31  40.0  285.0  274.704355
2014-04-30  40.0  285.0  282.099000
2014-05-31  40.0  285.0  288.787742
2014-06-30  40.0  285.0  294.042000
2014-07-31  40.0  285.0  295.739032
2014-08-31  40.0  285.0  295.084677
2014-09-30  40.0  285.0  293.141000
2014-10-31  40.0  285.0  288.132177
2014-11-30  40.0  285.0  279.517667
2014-12-31  40.0  285.0  277.252258
